In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the CSV file into a DataFrame
df_ibex = pd.read_csv('data/ibex.csv')

# Convert the 'date' column to datetime if it's not already in datetime format
df_ibex['Date'] = pd.to_datetime(df_ibex['Date'])

# Set the 'date' column as the index
df_ibex.set_index('Date', inplace=True)

# Drop 'Volume' and 'Adj Close' columns
df_ibex = df_ibex.drop(['Volume', 'Adj Close'], axis=1)

In [3]:
df_ibex

,Open,High,Low,Close
Date,,,,
2011-01-03,9899.400391,9993.599609,9850.500000,9888.299805
2011-01-04,9895.500000,9974.200195,9799.400391,9888.400391
2011-01-05,9877.500000,9882.400391,9599.299805,9801.400391
2011-01-06,9803.200195,9832.599609,9678.400391,9702.700195
2011-01-07,9682.400391,9699.400391,9497.799805,9560.700195
...,...,...,...,...
2019-12-23,9650.200195,9670.900391,9639.000000,9659.599609
2019-12-24,9632.099609,9661.799805,9607.799805,9661.799805
2019-12-27,9673.000000,9700.500000,9657.500000,9700.500000


In [4]:
# Define the split date
split_date = pd.to_datetime('2019-01-01')

# Split the data into train and test sets
df_ibex_train = df_ibex[df_ibex.index < split_date]
df_ibex_test = df_ibex[df_ibex.index >= split_date]

### **TEMA**

In [5]:
def ema(df, period):
    return df.ewm(span = period, min_periods = period).mean() 

In [6]:
def df_change(df, symbol):
    temp = pd.DataFrame(df['Close'])
    temp = temp.rename({'Close': symbol}, axis=1)
    return temp

In [28]:
def tema_strategy(df, symbol, short_period, mid_period, long_period):
    df = df_change(df, symbol)
    temp = df.copy()
    temp['ShortEMA'] = ema(df, short_period)
    temp['MidEMA'] = ema(df, mid_period)
    temp['LongEMA'] = ema(df, long_period)
    temp['Position'] = np.where((temp['ShortEMA'] > temp['MidEMA']) & (temp['ShortEMA'] > temp['LongEMA']), 1,
                                np.where((temp['ShortEMA'] < temp['MidEMA']) & (temp['ShortEMA'] < temp['LongEMA']), -1, 0))
    
    # Calculate log returns and strategy based on log returns
    temp['LogReturns'] = np.log(temp[symbol]) - np.log(temp[symbol].shift(1))
    temp['Strategy'] = temp.Position.shift(1) * temp['LogReturns']
    
    # Calculate profit based on price
    temp['PriceChange'] = temp[symbol] - temp[symbol].shift(1)
    temp['PriceStrategy'] = temp.Position.shift(1) * temp['PriceChange']
    
    temp['Trades'] = temp.Position.diff().fillna(0).abs()
    temp['Trades'] = temp['Trades'].replace(2, 1)
    temp['CLogReturns'] = temp['LogReturns'].cumsum().apply(np.exp)
    temp["CStrategy"] = temp['Strategy'].cumsum().apply(np.exp)
    temp['CPrice'] = -(df[symbol].iloc[0] - df[symbol])
    temp['CPriceStrategy'] = temp['PriceStrategy'].cumsum()
    
    return temp

In [9]:
from itertools import product
import heapq

def optimize_tema_parameters(df, symbol, short_range, mid_range, long_range):
    top_combinations = []
    
    for short_period, mid_period, long_period in product(short_range, mid_range, long_range):
        if short_period < mid_period < long_period:
            result = tema_strategy(df, symbol, short_period, mid_period, long_period)
            c_strategy = result['CStrategy'].iloc[-1]
            
            # Use a heap to maintain the top 5 combinations based on CStrategy
            if len(top_combinations) < 5:
                heapq.heappush(top_combinations, (c_strategy, (short_period, mid_period, long_period)))
            else:
                heapq.heappushpop(top_combinations, (c_strategy, (short_period, mid_period, long_period)))
    
    # Get the top 5 combinations with highest CStrategy values
    top_combinations = sorted(top_combinations, reverse=True)
    best_combinations = [combo[1] for combo in top_combinations]
    best_c_strategy = [combo[0] for combo in top_combinations]
    
    return best_combinations, best_c_strategy

In [21]:
# Define the parameter ranges
# short_range = range(2, 25)
# mid_range = range(5, 41)
# long_range = range(8, 81)

# test
short_range = range(2, 5)
mid_range = range(5, 7)
long_range = range(8, 12)

# Run the optimization
best_combinations, best_c_strategy = optimize_tema_parameters(df_ibex_train, 'IBEX', short_range, mid_range, long_range)

# Print the results
print("Best Combinations:")
for i, combination in enumerate(best_combinations):
    print(f"Combination {i+1}: {combination}")
print("Best CStrategy:", best_c_strategy)

Best Combinations:
Combination 1: (2, 5, 8)
Combination 2: (2, 6, 8)
Combination 3: (2, 5, 9)
Combination 4: (2, 6, 9)
Combination 5: (3, 6, 8)
Best CStrategy: [0.9723223629466213, 0.9476243764850367, 0.9244638122291597, 0.900981482100017, 0.859010805614707]


In [22]:
best_params = best_combinations[0]

In [29]:
tema_strategy(df_ibex_train, 'IBEX', best_params[0], best_params[1], best_params[2])

,IBEX,ShortEMA,MidEMA,LongEMA,Position,LogReturns,Strategy,PriceChange,PriceStrategy,Trades,CLogReturns,CStrategy,CPrice,CPriceStrategy
Date,,,,,,,,,,,,,,
2011-01-03,9888.299805,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-0.000000,NaN
2011-01-04,9888.400391,9888.375244,NaN,NaN,0,0.000010,0.000000,0.100586,0.000000,0.0,1.000010,1.000000,0.100586,0.000000
2011-01-05,9801.400391,9828.161884,NaN,NaN,0,-0.008837,-0.000000,-87.000000,-0.000000,0.0,0.991212,1.000000,-86.899414,0.000000
2011-01-06,9702.700195,9743.475244,NaN,NaN,0,-0.010121,-0.000000,-98.700195,-0.000000,0.0,0.981230,1.000000,-185.599609,0.000000
2011-01-07,9560.700195,9621.121699,9700.224392,NaN,0,-0.014743,-0.000000,-142.000000,-0.000000,0.0,0.966870,1.000000,-327.599609,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,8556.799805,8588.366075,8668.619282,8718.513686,-1,-0.004629,0.004629,-39.700195,39.700195,0.0,0.865346,0.970402,-1331.500000,82.001465
2018-12-24,8480.599609,8516.521765,8605.946057,8665.643891,-1,-0.008945,0.008945,-76.200195,76.200195,0.0,0.857640,0.979121,-1407.700195,158.201660
2018-12-27,8363.900391,8414.774182,8525.264169,8598.589780,-1,-0.013856,0.013856,-116.699219,116.699219,0.0,0.845838,0.992783,-1524.399414,274.900879


In [83]:
def calculate_profit_factor(strategy):
    positive_trades = strategy.loc[strategy > 0].sum()
    negative_trades = strategy.loc[strategy < 0].sum()
    profit_factor = abs(positive_trades / negative_trades)
    return profit_factor

In [84]:
for i, combination in enumerate(best_combinations):
    short_period, mid_period, long_period = combination
    
    # Run the strategy with the current combination
    strategy_result = tema_strategy(df_ibex_train, 'IBEX', short_period, mid_period, long_period)
    
    # Calculate the profit factor
    pf_factor = calculate_profit_factor(strategy_result['Strategy'])
    
    # Print the result
    print(f"Combination {i+1}: {combination}")
    print("Profit Factor:", pf_factor)
    print()

Combination 1: (24, 40, 80)
Profit Factor: 0.9563042976592117

Combination 2: (24, 40, 79)
Profit Factor: 0.9541157308911783

Combination 3: (24, 40, 78)
Profit Factor: 0.9539606804398535

Combination 4: (24, 40, 77)
Profit Factor: 0.953276900794885

Combination 5: (24, 40, 76)
Profit Factor: 0.9528592915700163



### **MACD**

In [76]:
def macd_strategy(df, symbol, short_period, long_period, signal_period):
    df = df_change(df, symbol)
    temp = df.copy()
    
    # Calculate MACD line
    temp['ShortEMA'] = ema(df, short_period)
    temp['LongEMA'] = ema(df, long_period)
    temp['MACD Line'] = temp['ShortEMA'] - temp['LongEMA']
    
    # Calculate signal line
    temp['Signal Line'] = ema(temp, signal_period)['MACD Line']
    
    # Generate trading signals
    temp['Position'] = np.where(temp['MACD Line'] > temp['Signal Line'], 1,
                                np.where(temp['MACD Line'] < temp['Signal Line'], -1, 0))
    
    # Calculate log returns and strategy returns
    temp['LogReturns'] = np.log(temp[symbol]) - np.log(temp[symbol]).shift(1)
    temp['Strategy'] = temp['Position'].shift(1) * temp['LogReturns']
    
    # Calculate profit based on price
    temp['PriceChange'] = temp[symbol] - temp[symbol].shift(1)
    temp['PriceStrategy'] = temp.Position.shift(1) * temp['PriceChange']
    
    temp['Trades'] = temp.Position.diff().fillna(0).abs()
    temp['Trades'] = temp['Trades'].replace(2, 1)
    temp['CLogReturns'] = temp['LogReturns'].cumsum().apply(np.exp)
    temp["CStrategy"] = temp['Strategy'].cumsum().apply(np.exp)
    temp['CPrice'] = -(df[symbol].iloc[0] - df[symbol])
    temp['CPriceStrategy'] = temp['PriceStrategy'].cumsum()
    
    return temp

In [31]:
def optimize_macd_parameters(df, symbol, short_range, long_range, signal_range):
    top_combinations = []
    
    for short_period, long_period, signal_period in product(short_range, long_range, signal_range):
        if short_period < long_period:# and short_period < signal_period and long_period < signal_period:
            result = macd_strategy(df, symbol, short_period, long_period, signal_period)
            c_strategy = result['CStrategy'].iloc[-1]
            
            # Use a heap to maintain the top 5 combinations based on CStrategy
            if len(top_combinations) < 5:
                heapq.heappush(top_combinations, (c_strategy, (short_period, long_period, signal_period)))
            else:
                heapq.heappushpop(top_combinations, (c_strategy, (short_period, long_period, signal_period)))
    
    # Get the top 5 combinations with highest CStrategy values
    top_combinations = sorted(top_combinations, reverse=True)
    best_combinations = [combo[1] for combo in top_combinations]
    best_c_strategy = [combo[0] for combo in top_combinations]
    
    return best_combinations, best_c_strategy

In [32]:
# Define the parameter ranges
# short_range = range(2, 10)
# long_range = range(5, 35)
# signal_range = range(5, 20)

# test
short_range = range(2, 5)
long_range = range(5, 7)
signal_range = range(5, 10)

# Run the optimization
best_combinations, best_c_strategy = optimize_macd_parameters(df_ibex_train, 'IBEX', short_range, long_range, signal_range)

# Print the best combinations and CStrategy values
print("Best Combinations:")
for i, combination in enumerate(best_combinations):
    print(f"Combination {i+1}: {combination}")
print("Best CStrategy:", best_c_strategy)

Best Combinations:
Combination 1: (2, 6, 6)
Combination 2: (2, 5, 7)
Combination 3: (3, 5, 5)
Combination 4: (2, 5, 8)
Combination 5: (2, 6, 7)
Best CStrategy: [2.0826343248835086, 2.07414619396561, 1.8325203448908154, 1.547947186325397, 1.4387398803626734]


In [34]:
best_params = best_combinations[0]

In [35]:
macd_strategy(df_ibex_train, 'IBEX', best_params[0], best_params[1], best_params[2])

,IBEX,ShortEMA,LongEMA,MACD Line,Signal Line,Position,LogReturns,Strategy,PriceChange,PriceStrategy,Trades,CLogReturns,CStrategy,CPrice,CPriceStrategy
Date,,,,,,,,,,,,,,,
2011-01-03,9888.299805,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-0.000000,NaN
2011-01-04,9888.400391,9888.375244,NaN,NaN,NaN,0,0.000010,0.000000,0.100586,0.000000,0.0,1.000010,1.000000,0.100586,0.000000
2011-01-05,9801.400391,9828.161884,NaN,NaN,NaN,0,-0.008837,-0.000000,-87.000000,-0.000000,0.0,0.991212,1.000000,-86.899414,0.000000
2011-01-06,9702.700195,9743.475244,NaN,NaN,NaN,0,-0.010121,-0.000000,-98.700195,-0.000000,0.0,0.981230,1.000000,-185.599609,0.000000
2011-01-07,9560.700195,9621.121699,NaN,NaN,NaN,0,-0.014743,-0.000000,-142.000000,-0.000000,0.0,0.966870,1.000000,-327.599609,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,8556.799805,8588.366075,8687.297424,-98.931348,-62.391480,-1,-0.004629,0.004629,-39.700195,39.700195,0.0,0.865346,2.056093,-1331.500000,7062.391602
2018-12-24,8480.599609,8516.521765,8628.240905,-111.719141,-76.485098,-1,-0.008945,0.008945,-76.200195,76.200195,0.0,0.857640,2.074567,-1407.700195,7138.591797
2018-12-27,8363.900391,8414.774182,8552.715044,-137.940862,-94.043887,-1,-0.013856,0.013856,-116.699219,116.699219,0.0,0.845838,2.103513,-1524.399414,7255.291016


In [36]:
def calculate_profit_factor(strategy):
    positive_trades = strategy.loc[strategy > 0].sum()
    negative_trades = strategy.loc[strategy < 0].sum()
    profit_factor = abs(positive_trades / negative_trades)
    return profit_factor

In [37]:
# Calculate and print the profit factor for each best combination
for i, combination in enumerate(best_combinations):
    short_period, long_period, signal_period = combination
    
    # Run the strategy with the current combination
    strategy_result = macd_strategy(df_ibex_train, 'IBEX', short_period, long_period, signal_period)
    
    # Calculate the profit factor
    pf_factor = calculate_profit_factor(strategy_result['Strategy'])
    
    # Print the result
    print(f"Combination {i+1}: {combination}")
    print("Profit Factor:", pf_factor)
    print()

Combination 1: (2, 6, 6)
Profit Factor: 1.0766055374858183

Combination 2: (2, 5, 7)
Profit Factor: 1.0761628492835382

Combination 3: (3, 5, 5)
Profit Factor: 1.062788710723144

Combination 4: (2, 5, 8)
Profit Factor: 1.0449952243872622

Combination 5: (2, 6, 7)
Profit Factor: 1.037320375059354



In [98]:
def calculate_metrics(df):
    # Calculate number of trades
    num_trades = df['Trades'].sum()
    
    # Calculate net profit of the trades
    net_profit = df['PriceStrategy'].sum()
    
    # Calculate profit factor
    profit_factor = calculate_profit_factor(df['Strategy'])
    
    # Calculate average net of each trade
    avg_net_trade = net_profit / num_trades
    
    # Calculate maximum drawdown (price)
    max_drawdown = (df['CPriceStrategy'] - df['CPriceStrategy'].cummax()).min()
    
     # Calculate number of winning trades
    num_winning_trades = 0
    in_position = False
    for i in range(len(df)):
        if df['Position'][i] != 0:
            in_position = True
        if in_position and df['PriceStrategy'][i] < 0:
            num_winning_trades += 1
            in_position = False
    
    # Calculate Percent Profitable (PP)
    pp = num_winning_trades / num_trades * 100 if num_trades > 0 else 0

    
    # Create a DataFrame to store the metrics
    metrics_df = pd.DataFrame({
        'Number of Trades': [num_trades],
        'Net Profit': [net_profit],
        'Profit Factor': [profit_factor],
        'Average Net per Trade': [avg_net_trade],
        'Maximum Drawdown': [max_drawdown],
        'Percent Profitable (PP)': [pp]
    }, index=['Strategy'])
    
    return metrics_df

In [101]:
calculate_metrics(macd_strategy(df_ibex_test, 'IBEX', best_params[0], best_params[1], best_params[2]))

,Number of Trades,Net Profit,Profit Factor,Average Net per Trade,Maximum Drawdown,Percent Profitable (PP)
Strategy,60.0,495.306641,1.072903,8.255111,-772.899414,201.666667


In [103]:
macd_strategy(df_ibex_test, 'IBEX', best_params[0], best_params[1], best_params[2])

,IBEX,ShortEMA,LongEMA,MACD Line,Signal Line,Position,LogReturns,Strategy,PriceChange,PriceStrategy,Trades,CLogReturns,CStrategy,CPrice,CPriceStrategy
Date,,,,,,,,,,,,,,,
2019-01-02,8550.000000,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,-0.000000,NaN
2019-01-03,8523.299805,8529.974854,NaN,NaN,NaN,0,-0.003128,-0.000000,-26.700195,-0.000000,0.0,0.996877,1.000000,-26.700195,0.000000
2019-01-04,8737.799805,8673.853666,NaN,NaN,NaN,0,0.024855,0.000000,214.500000,0.000000,0.0,1.021965,1.000000,187.799805,0.000000
2019-01-07,8776.299805,8743.004810,NaN,NaN,NaN,0,0.004396,0.000000,38.500000,0.000000,0.0,1.026468,1.000000,226.299805,0.000000
2019-01-08,8847.299805,8812.822120,NaN,NaN,NaN,0,0.008057,0.000000,71.000000,0.000000,0.0,1.034772,1.000000,297.299805,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,9659.599609,9658.567121,9619.675338,38.891783,54.920219,-1,-0.001645,0.001645,-15.900391,15.900391,0.0,1.129778,1.040002,1109.599609,384.907227
2019-12-24,9661.799805,9660.722243,9631.710900,29.011344,47.517683,-1,0.000228,-0.000228,2.200195,-2.200195,0.0,1.130035,1.039765,1111.799805,382.707031
2019-12-27,9700.500000,9687.240748,9651.364928,35.875820,44.191436,-1,0.003997,-0.003997,38.700195,-38.700195,0.0,1.134561,1.035617,1150.500000,344.006836


In [83]:
asag = macd_strategy(df_ibex_test, 'IBEX', best_params[0], best_params[1], best_params[2])